In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning

## Homework 7: Word embeddings and topic modeling
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [11]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Instructions
rubric={points}

You will earn points for following these instructions and successfully submitting your work on Gradescope.  

### Group wotk instructions

**You may work with a partner on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2.
  
- Use group work as an opportunity to collaborate and learn new things from each other.
- Be respectful to each other and make sure you understand all the concepts in the assignment well.
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline.
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).
- If you would like to use late tokens for the homework, all group members must have the necessary late tokens available. Please note that the late tokens will be counted for all members of the group.   


### General submission instructions

- Please **read carefully
[Use of Generative AI policy](https://ubc-cs.github.io/cpsc330-2025W1/syllabus.html#use-of-generative-ai-in-the-course)** before starting the homework assignment.
- **Run all cells before submitting:** Go to `Kernel -> Restart Kernel and Clear All Outputs`, then select `Run -> Run All Cells`. This ensures your notebook runs cleanly from start to finish without errors.
  
- **Submit your files on Gradescope.**  
   - Upload only your `.ipynb` file **with outputs displayed** and any required output files.
     
   - Do **not** submit other files from your repository.  
   - If you need help, see the [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/).  
- **Check that outputs render properly.**  
   - Make sure all plots and outputs appear in your submission.
     
   - If your `.ipynb` file is too large and doesn't render on Gradescope, also upload a PDF or HTML version so the TAs can view your work.  
- **Keep execution order clean.**  
   - Execution numbers must start at "1" and increase in order.
     
   - Notebooks without visible outputs may not be graded.  
   - Out-of-order or missing execution numbers may result in mark deductions.  
- **Follow course submission guidelines:** Review the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html) for detailed guidance on completing and submitting assignments.
   
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl.

A number of pre-trained word embeddings are available out there. Some popular ones are:

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf)
    * published by Stanford University
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html)
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below.

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [12]:
# !pip install gensim

In [13]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [14]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [15]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model.

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings.

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [16]:
WORDS = ["climbing", "university", "fashion", "banking"]

In [17]:
def show_similar(word, model=glove_wiki_vectors, topn=10):
    # Gensim 4.x vocabulary accessor:
    if word not in model.key_to_index:
        print(f" '{word}' not in vocabulary. Try a different word.")
        return None

    sims = model.most_similar(word, topn=topn)
    # Pretty print
    print(f"\nTop {topn} words similar to: '{word}'")
    for w, s in sims:
        print(f"  {w:<20}  {s:.3f}")
    return sims


In [18]:
all_results = {}
for w in WORDS:
    res = show_similar(w, topn=10)
    if res is not None:
        all_results[w] = res



Top 10 words similar to: 'climbing'
  climb                 0.785
  jumping               0.727
  climbs                0.687
  hiking                0.665
  climbed               0.649
  biking                0.636
  trekking              0.620
  falling               0.616
  walking               0.608
  climbers              0.608

Top 10 words similar to: 'university'
  college               0.829
  harvard               0.816
  yale                  0.811
  professor             0.810
  graduate              0.799
  faculty               0.779
  princeton             0.763
  institute             0.760
  school                0.755
  stanford              0.742

Top 10 words similar to: 'fashion'
  designer              0.717
  designers             0.694
  fashions              0.688
  clothing              0.657
  chic                  0.653
  style                 0.642
  fashionable           0.638
  couture               0.638
  art                   0.636
  look            

In [21]:
rows = []
for w, pairs in all_results.items():
    for sim_word, score in pairs[:5]:
        rows.append({"query_word": w, "similar_word": sim_word, "cosine": score})

df_sim = pd.DataFrame(rows)
df_sim


,query_word,similar_word,cosine
0,climbing,climb,0.784659
1,climbing,jumping,0.726908
2,climbing,climbs,0.687022
3,climbing,hiking,0.664511
4,climbing,climbed,0.649294
5,university,college,0.829421
6,university,harvard,0.815603
7,university,yale,0.811380
8,university,professor,0.810378
9,university,graduate,0.799300


In [22]:
present = [w for w in WORDS if w in glove_wiki_vectors.key_to_index]
embs = np.stack([glove_wiki_vectors[w] for w in present])

# cosine similarity
from numpy.linalg import norm
S = (embs @ embs.T) / (norm(embs, axis=1)[:, None] * norm(embs, axis=1)[None, :])

pd.DataFrame(S, index=present, columns=present).round(3)


,climbing,university,fashion,banking
climbing,1.000,0.078,0.217,0.187
university,0.078,1.000,0.309,0.271
fashion,0.217,0.309,1.000,0.210
banking,0.187,0.271,0.210,1.000


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [23]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [24]:
results = []
for w1, w2 in word_pairs:
    # check if both words exist in vocab
    if w1 in glove_wiki_vectors.key_to_index and w2 in glove_wiki_vectors.key_to_index:
        sim = glove_wiki_vectors.similarity(w1, w2)
        results.append((w1, w2, sim))
    else:
        results.append((w1, w2, None))  # mark missing words

df_pairs = pd.DataFrame(results, columns=["word1", "word2", "cosine_similarity"])
df_pairs


,word1,word2,cosine_similarity
0,coast,shore,0.700027
1,clothes,closet,0.546276
2,old,new,0.643249
3,smart,intelligent,0.755273
4,dog,cat,0.879807
5,tree,lawyer,0.076719


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not.
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True.

In [25]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [26]:
results = []

for word in test_words:
    in_vocab = word in glove_wiki_vectors.key_to_index
    results.append((word, in_vocab))


df_vocab = pd.DataFrame(results, columns=["word", "in_vocabulary"])
df_vocab


,word,in_vocabulary
0,covididiot,False
1,fomo,False
2,frenemies,True
3,anthropause,False
4,photobomb,False
5,selfie,False
6,pxg,False
7,pacg,False
8,cct,True
9,escc,True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data.

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this.
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models.

In [27]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [29]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [30]:
glove_wiki_vectors.similarity("aboriginal", "success")

np.float32(0.14283238)

In [31]:
glove_wiki_vectors.similarity("white", "success")

np.float32(0.35182396)

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [32]:
# gender bias in occupations
analogy("man", "engineer", "woman")

man : engineer :: woman : ?


,Analogy word,Score
0,technician,0.662020
1,educator,0.611515
2,engineers,0.592235
3,surgeon,0.588020
4,contractor,0.587330
5,engineering,0.568600
6,nurse,0.556342
7,chemist,0.554719
8,pioneer,0.546008
9,physician,0.545960


In [33]:
analogy("france", "wine", "japan")

france : wine :: japan : ?


,Analogy word,Score
0,seafood,0.598042
1,beverage,0.547623
2,fruit,0.547266
3,sushi,0.544637
4,apples,0.544177
5,japanese,0.530488
6,fruits,0.529355
7,tea,0.527423
8,products,0.516392
9,packaged,0.512219


In [34]:
# nationality / profession analogies
analogy("italy", "pasta", "mexico")

italy : pasta :: mexico : ?


,Analogy word,Score
0,beans,0.641126
1,tortillas,0.624782
2,soy,0.624413
3,tortilla,0.621994
4,tomato,0.621223
5,corn,0.610944
6,sauce,0.603141
7,tacos,0.602411
8,chicken,0.599965
9,salad,0.597623


In [35]:
analogy("man", "driving", "woman")

man : driving :: woman : ?


,Analogy word,Score
0,car,0.635288
1,driver,0.627889
2,speeding,0.617300
3,vehicle,0.608771
4,drivers,0.599836
5,riding,0.591213
6,drunk,0.582134
7,truck,0.578620
8,driven,0.566886
9,cars,0.566099


In [36]:
analogy("man", "programmer", "woman")

man : programmer :: woman : ?


,Analogy word,Score
0,educator,0.585333
1,programmers,0.573053
2,linguist,0.543201
3,technician,0.537336
4,freelance,0.536212
5,animator,0.533937
6,translator,0.533419
7,software,0.496662
8,psychotherapist,0.495662
9,technologist,0.488051


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1. From the analogies in 1.4, it's clear that even embeddings trained on Wikipedia still carry some social and cultural biases.
For example, in the analogy “man, doctor, woman”, the top word returned was “nurse”, and similarly “man, engineer, woman” also leaned toward roles like “educator” or “nurse”.
These results show gender stereotypes where the model associates women more with caregiving or teaching roles and men with technical or higher-status professions.
In the nationality analogies like “france, wine, japan” and “italy, pasta, mexico”, the embeddings linked countries to stereotypical cultural items such as “sushi” or “tacos.”
This suggests that the model captures narrow or oversimplified cultural associations.
Overall, these examples show that pre-trained embeddings reflect the biases and frequency patterns found in the text they were trained on, even if that text (like Wikipedia) aims to be neutral.

2. Using biased embeddings in real applications could lead to harmful outcomes.
For instance, if a resume screening model or recommendation system relies on such embeddings, it might unintentionally favor men for technical roles or associate certain ethnicities or cultures with lower success.
This shows why it’s important to be aware of, test for, and mitigate bias before deploying NLP systems in the real world.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling

The goal of topic modeling is discovering high-level themes in a large collection of texts.

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model.

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics.

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories.

In [37]:
from sklearn.datasets import fetch_20newsgroups

In [38]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [39]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling.

**Your tasks:**

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md.

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include:
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM".
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing.

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy.

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps.

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook.



In [40]:
# !python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 20.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [41]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [46]:
df.head()

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast


In [80]:
def preprocess(token):
    if token.like_url:
      return "URL"
    elif token.like_email:
      return "EMAIL"
    elif token.like_num:
      return "NUM"
    elif token.is_punct or token.is_stop or token.is_oov:
      # delete token if it's a punctutation or stop word or a word not in vocab
      return ""
    else:
      return token.lemma_

In [81]:
from tqdm import tqdm

for i in tqdm(range(len(df)), desc="Processing texts"):
  curr_text = df.iloc[i].text
  cleaned_text = curr_text.replace("/", " ").replace("\\", " ").replace("\n", " ").replace("\r", " ")
  doc = nlp(cleaned_text)
  processed_text = ""
  for token in doc:
    # process each token in the text, then add it back into df in text_pp column
    processed_text += preprocess(token) + " "
  df.at[i, "text_pp"] = processed_text

Processing texts: 100%|██████████| 4563/4563 [01:47<00:00, 42.29it/s]


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 4

I decided on n_topics = 8 becuase we fetched 8 categories from the newsgroup dataset. Therefore, I think that having 8 topics would be appropriate to separate the topics accordingly.

In [82]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

n_topics = 8 # number of topics
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)

vec = CountVectorizer(stop_words="english")
X_processed = vec.fit_transform(df["text_pp"])

lda.fit(X_processed)
document_topics = lda.transform(X_processed)


In [83]:

feature_names = np.array(vec.get_feature_names_out())
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]

In [86]:
lda.components_.shape

(8, 24115)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class).

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 5

_Type your answer here, replacing this text._

In [ ]:
# !pip install mglearn

In [85]:
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
henrik        god           num           num           num           
think         num           url           gun           god           
bm            jesus         email         law           think         
know          sin           file          state         know          
want          islam         image         people        people        
kk            spirit        use           weapon        believe       
armenians     son           window        right         good          
armenia       father        program       firearm       like          
let           book          available     file          time          
guy           mary          run           control       thing         


topic 5       topic 6       topic 7       
--------      --------      --------      
num           num           num           
entry         peo

Topic 0: Armenia related texts

Topic 1: Christianity

Topic 2: Tech/Programming

Topic 3: Gun Control

Topic 4: Beliefs/Religion

Topic 5: Programming

Topic 6: Ethnicities

Topic 7: Sports


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`.

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [92]:
for i in range(5):
  print(f"Document {i+1} distribution: {document_topics[i]}")
  print(f"Document {i+1} assigned Topic {np.argmax(document_topics[i])}\n")

Document 1 distribution: [0.0021196  0.00212031 0.00212148 0.62066506 0.31900444 0.00212035
 0.00212207 0.04972668]
Document 1 assigned Topic 3

Document 2 distribution: [0.02963264 0.00156353 0.89857411 0.00156392 0.00156543 0.00156467
 0.00156507 0.06397062]
Document 2 assigned Topic 2

Document 3 distribution: [0.00219483 0.00219434 0.35654498 0.00219551 0.56286172 0.00219474
 0.00219617 0.06961771]
Document 3 assigned Topic 4

Document 4 distribution: [0.00480883 0.0048134  0.47746681 0.49365619 0.0048163  0.00481062
 0.00481634 0.00481151]
Document 4 assigned Topic 3

Document 5 distribution: [0.00284345 0.00284327 0.002844   0.1947403  0.00284453 0.0028436
 0.78819915 0.00284171]
Document 5 assigned Topic 6



<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems.
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. 

content-based filtering works by building a "profile" for each user, then  training a model(usually regression), in order to get coefficients/weights for each available feature. When we trained the model, we can then predict the rating for a new item for the same user.

2. 

Recommender systems tend to reflect biases heavily, because its rating predictions are either based on past ratings of other users, or the same user's past ratings. This can lead to biases or unfairness in the real world, where certain items or individuals are preferred and rated higher over others.

Another possible negative consequence is that recommender systems are more likely to recommend items based on a user's/other users' previous ratings. This can lead to a feedback loop & echo chamber of recommending ONLY similar/popular content, meaning fresh/unpopular content will almost never be recommended.

3. 

Transfer learning is when we use a pre-trained model and fine-tune it to solve our task. This is a common approach becuase training a neural network from scratch requires an extremely large dataset, and is very costly.



<!-- END QUESTION -->

<br><br><br><br>

Before submitting your assignment, please make sure you have followed all the instructions in the Submission Instructions section at the top.

Here is a quick checklist before submitting:

- [ ] Restart kernel, clear outputs, and run all cells from top to bottom.  
- [ ] `.ipynb` file runs without errors and contains all outputs.  
- [ ] Only `.ipynb` and required output files are uploaded (no extra files).  
- [ ] Execution numbers start at **1** and are in order.  
- [ ] If `.ipynb` is too large and doesn't render on Gradescope, also upload a PDF/HTML version.  
- [ ] Reviewed the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html).  

![](img/eva-well-done.png)